In [1]:
import numpy as np
import random
from poker_tools import range_simulate, hand_simulate
import copy

In [2]:
##initialize
cards = set()
suits = {"Spade", "Diamond", "Heart", "Club"}
numbers = {"2":0, "3":1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7, "10":8, "J":9, "Q":10, "K":11,"A":12}

for number in numbers:
    for suit in suits:
        cards.add((suit, number))

In [7]:
set_direct = {
    "S":{("A", "K"), ("A", "Q")},
    "P":{"2", "5", "6", "A"},
    "O":{("A", "Q"), ("A", "10")}
}

hero = {("Club", "J"), ("Diamond", "Q")}
villains = {"Villain1":{}}
flop = set()
equity, win_dict = range_simulate(hero, villains, flop, cards, 100000)
print(equity)
print(win_dict)

0.57752
{'Straight Flush': 0.0, '4 of a Kind': 0.0, 'Full House': 0.04, 'Flush': 0.04, 'Straight': 0.1, '3 of a Kind': 0.07, 'Two Pair': 0.31, 'Pair': 0.37, 'High Card': 0.06}


In [15]:
hero = {("Heart", "J"), ("Heart", "10")}
villains = {"Villain":{("Club", "2"),("Diamond", "2")}}
flop = set()

equity, win_dict = hand_simulate(hero, villains, flop, cards, 100000)
print(equity)
print(win_dict)

0.54036
{'Straight Flush': 0.0, '4 of a Kind': 0.01, 'Full House': 0.1, 'Flush': 0.08, 'Straight': 0.1, '3 of a Kind': 0.13, 'Two Pair': 0.34, 'Pair': 0.24, 'High Card': 0.0}


In [ ]:
'''
Organization: 
1) Add players (Poker Bot Class)
2) 

'''

In [2]:
class PokerBotTemplate:
    def __init__(self, name):
        self.name = name
        self.saved_state = {}
        pass
    
    def initialize_strategy(self, state):
        pass
    
    def play(self, state):
        '''
        return tuple (Action, Amount)
        '''
        return ("CheckAction", state["call_amount"])

In [3]:
class FishPlayer(PokerBotTemplate):
    def __init__(self, name):
        super().__init__(name)

In [4]:
class RaisePlayer(PokerBotTemplate):
    def __init__(self, name):
        super().__init__(name)
    
    def play(self, state):
        if state["stack_size"][self.name] < state["min_raise"]:
            return ("CheckAction", state["call_amount"])
        
        return ("RaiseAction", state["min_raise"])

In [27]:
class CheckOrFoldPlayer(PokerBotTemplate):
    def __init__(self, name):
        super().__init__(name)
    
    def play(self, state):
        check = random.random() > 0.5
        if check:
            return ("CheckAction", 0)
        return ("FoldAction", 0)

In [138]:
class PokerGame:
    def __init__(self, max_players = 10, starting_pot = 1000, blinds = (5, 10)):
        
        assert(type(max_players) == int and max_players > 0 and max_players <= 10)
        assert(type(starting_pot) == int and starting_pot > 0)
        assert(len(blinds) == 2 and type(blinds[0]) == int and type(blinds[1]) == int)
        assert(blinds[0] > 0 and blinds[1] > 0 and blinds[1] > blinds[0])
        
        #sb is first, bb is second, third = under the gun
        self.round = 1
        self.player_list = []
        self.player_names = {}
        
        cards = set()
        suits = {"Spade", "Diamond", "Heart", "Club"}
        numbers = {"2":0, "3":1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7, "10":8, "J":9, "Q":10, "K":11,"A":12}

        for number in numbers:
            for suit in suits:
                cards.add((suit, number))
        
        self.cards = cards
        self.max_players = max_players
        self.starting_pot = starting_pot
        #first is small blind, second is big blind
        self.blinds = blinds
        
        self.has_started = False
    
    def add_player(self, poker_bot):
        if self.has_started:
            print("Cannot add player, match has started")
            return
        
        assert(issubclass(poker_bot.__class__, PokerBotTemplate))
        assert(poker_bot.name not in self.player_names)
        if len(self.player_list) == self.max_players:
            print("No more players can join")
            return
        self.player_list.append(poker_bot)
        self.player_names[poker_bot.name] = 0
    
    def initialize_game(self):
        self.has_started = True
        for player in self.player_list:
            self.player_names[player.name] = [player, self.starting_pot]
    
    def return_results(self):
        if not self.has_started:
            print("Game has not started yet")
            return
        new_result = {}
        for player in self.player_names:
            new_result[player] = self.player_names[player][1]
        return new_result
    
    def play_round(self):
        cards = self.cards.copy()
        
        #draw hands
        hands_ref = {}
        hands = random.sample([*cards], 2 * len(self.player_list))
        
        index = 0
        stacks = {}
        for player in self.player_names:
            hands_ref[player] = {hands[index], hands[index+1]}
            stacks[player] = self.player_names[player][1]
            index += 2
        
        state = {"round":self.round, "stacks":stacks}
        for player in self.player_names:
            self.player_names[player][0].initialize_strategy(copy.deepcopy(state))
            
        player_list = self.player_list.copy()
        
        self.round += 1
        preflop_state = self.play_preflop(hands_ref, state, player_list)
        
        return
    
    def play_preflop(self, hands_ref, state, player_list):
        amount_call = {}
        stacks = {}
        blinds = self.blinds
        
        for player in self.player_names:
            amount_call[player] = blinds[1]
            stacks[player] = self.player_names[player][1]
        
        completed_betting = False
        num_iter = 0
        i = 1 if len(player_list) == 2 else 2
        game_history = []
        all_in = False
        
        pot = 0
        
        min_raise = blinds[1]
        
        if stacks[player_list[0].name] < blinds[0]:
            all_in = True
            amount = stacks[player_list[0].name]
            stacks[player_list[0].name] = 0
            pot += amount
            amount_call[player_list[0].name] = 0
        else: 
            pot += blinds[0]
            stacks[player_list[0].name] -= blinds[0]
            amount_call[player_list[0].name] -= blinds[0]
        
        if stacks[player_list[1].name] < blinds[1]:
            all_in = True
            amount = stacks[player_list[1].name][1]
            stacks[player_list[1].name] = 0
            pot += amount
        else:
            pot += blinds[1]
            stacks[player_list[1].name] -= blinds[1]     
        amount_call[player_list[1].name] = 0
        
        while not completed_betting:
            name = player_list[i].name
            forced_all_in = amount_call[name] > stacks[name]
            call_amount = min(amount_call[name], stacks[name])
            
            player_state = {
                "prog":"preflop",
                "position":i,
                "all_in":all_in,
                "game_history":game_history.copy(),
                "stack_size":stacks,
                "hand":hands_ref[name],
                "pot":pot,
                "call_amount":call_amount,
                "min_raise": min_raise
            }
            action = player_list[i].play(player_state)
            if all_in:
                if action[0] == "CheckAction":
                    stacks[name] -= call_amount
                    amount_call[name] = 0
                    game_history.append((name, "CheckAction", "Preflop", min_raise))
                    pot += call_amount
                    num_iter += 1
                    i = 0 if i + 1 >= len(player_list) else i + 1
                else:
                    player_list.pop(i)
                    i = 0 if i >= len(player_list) else i
                    game_history.append((name, "FoldAction", "Preflop", 0))
            else:
                if action[0] == "CheckAction":
                    stacks[name] -= call_amount
                    game_history.append((name, "CheckAction", "Preflop", min_raise))
                    num_iter += 1
                    amount_call[name] = 0
                    pot += call_amount
                    i = 0 if i + 1 >= len(player_list) else i + 1
                    if forced_all_in:
                        all_in = True
                elif action[0] == "RaiseAction":
                    if action[1] + call_amount > stacks[name] or action[1] < min_raise:
                        player_list.pop(i)
                        i = 0 if i + 1 >= len(player_list) else i + 1
                        game_history.append((name, "FoldAction", "Preflop", 0))
                    else:
                        min_raise = action[1] * 2
                        pot += action[1] + call_amount
                        num_iter = 1
                        game_history.append((name, "RaiseAction", "Preflop", action[1]))
                        stacks[name] -= (action[1] + call_amount)
                        if stacks[name] == 0:
                            all_in = True
                        i = 0 if i + 1 >= len(player_list) else i + 1
                        for player in amount_call:
                            amount_call[player] += action[1]
                else:
                    player_list.pop(i)
                    i = 0 if i >= len(player_list) else i
                    game_history.append((name, "FoldAction", "Preflop", 0))
            if num_iter == len(player_list) or len(player_list) == 1:
                completed_betting = True
        
        final_state = {}
        final_state["stacks"] = stacks
        final_state["game_history"] = game_history
        final_state["pot"] = pot
        final_state["player_list"] = player_list
        final_state["min_raise"] = min_raise
        final_state["all_in"] = all_in
        return final_state
        
    def play_postflop(self):
        pass
    
    def play_turn(self):
        pass
    
    def play_river(self):
        pass
    
    def play_showdown(self):
        pass

In [139]:
Game = PokerGame(max_players = 5)
Raise1 = RaisePlayer("Raise1")
CheckFold1 = CheckOrFoldPlayer("Fold1")
Raise2 = RaisePlayer("Raise2")
Fish1 = FishPlayer("Derrick")
Fish2 = FishPlayer("John")
Game.add_player(CheckFold1)
Game.add_player(Fish1)
#Game.add_player(Fish2)
#Game.add_player(Raise1)
#Game.add_player(Raise2)
Game.initialize_game()
Game.play_round()

In [122]:
dict_er = {"1":2, "4":5, "5":6}
sum(dict_er.values())

13